<a href="https://colab.research.google.com/github/psukphranee/Machine-Learning-Projects/blob/master/%5BKeras%5D_Implement_ResNet_34_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras

In [18]:
class ResidualUnit(keras.layers.Layer):

  def __init__(self, filters, strides=1, activation="relu", **kwargs):

    super().__init__(**kwargs)
    self.activation = keras.activations.get(activation)
    self.main_layers = [
        keras.layers.Conv2D(filters, 3, strides=strides, padding="same", use_bias=False),
        keras.layers.BatchNormalization(),
        self.activation,
        keras.layers.Conv2D(filters, 3, strides=1, padding="same", use_bias=False),
        keras.layers.BatchNormalization()
    ]

    self.skip_layers = []
    if strides > 1:
      self.skip_layers = [
          keras.layers.Conv2D(filters, 1, strides=strides, padding="same", use_bias=False),
        keras.layers.BatchNormalization()
      ]

  def call(self, inputs):

    Z = inputs

    #pass thru module
    for layer in self.main_layers:
      Z = layer(Z)

    skip_Z = inputs
    for layer in self.skip_layers:
      skip_Z = layer(skip_Z)

    return self.activation(Z + skip_Z)

In [19]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(64, kernel_size=7, strides=2,
                        input_shape=[28, 28, 1]))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding="SAME"))
prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    stridez = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters, strides=stridez))
    prev_filters = filters
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10, activation="softmax"))

In [20]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_26 (Conv2D)          (None, 11, 11, 64)        3200      
                                                                 
 batch_normalization_26 (Ba  (None, 11, 11, 64)        256       
 tchNormalization)                                               
                                                                 
 activation_8 (Activation)   (None, 11, 11, 64)        0         
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 residual_unit_12 (Residual  (None, 6, 6, 64)          74240     
 Unit)                                                           
                                                     